En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [1]:
%reload_ext autoreload
%autoreload 2

print("IPython autoreload activated")

IPython autoreload activated


In [2]:
import os

from google.colab import auth
auth.authenticate_user()


In [3]:
from google.cloud import bigquery

# Crear una instancia del cliente
client = bigquery.Client("tw-techdash")

In [4]:
!pip install memory-profiler



In [5]:
# Local file paths (consider user input/environment variables for flexibility)
MOUNT_POINT = os.environ['LC_MOUNT_POINT'] = "/content/drive"
SOURCE_PATH = os.environ['LC_SOURCE_PATH'] = "/content/drive/Othercomputers/My MacBook Pro/challenge_DE"


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
# Mount Google Drive (optional, depending on usage context)
# Uncomment the following line if mounting is needed:
drive.mount(MOUNT_POINT, force_remount=True)

# Change directory using a more explicit method
target_directory = os.path.join(MOUNT_POINT, SOURCE_PATH, "src")
os.chdir(target_directory)

# Confirm the current directory:
print(f"Changed directory to: {os.getcwd()}")  # Use os.getcwd() for accuracy

Mounted at /content/drive
Changed directory to: /content/drive/Othercomputers/My MacBook Pro/challenge_DE/src


In [8]:
query = """
WITH datos AS (
  SELECT
    FORMAT_TIMESTAMP('%Y-%m-%d', TIMESTAMP(date)) AS fecha_formateada,
    user.username as usuario
  FROM
    `tw-techdash.de_challenge_latam.tweets_latam`
),

conteo_por_fecha AS (
  SELECT
    fecha_formateada,
    COUNT(usuario) AS num_tweets
  FROM
    datos
  GROUP BY
    fecha_formateada
  ORDER BY
    num_tweets DESC
  LIMIT 10
),

usuario_top_por_fecha AS (
  SELECT
    fecha_formateada,
    usuario,
    COUNT(usuario) AS num_tweets_por_usuario,
    RANK() OVER (PARTITION BY fecha_formateada ORDER BY COUNT(usuario) DESC) AS rank
  FROM
    datos
  WHERE
    fecha_formateada IN (SELECT fecha_formateada FROM conteo_por_fecha)
  GROUP BY
    fecha_formateada, usuario
)

SELECT
  u.usuario AS top_user,
  u.fecha_formateada as date,
FROM
  usuario_top_por_fecha u
JOIN
  conteo_por_fecha c
ON
  u.fecha_formateada = c.fecha_formateada
WHERE
  u.rank = 1
ORDER BY
  c.num_tweets DESC;

"""

query_job = client.query(query)

query_job.result()

results = query_job.result()

output = [(row.top_user, row.date) for row in results]

print("QUERY 1")
print ("Las top 10 fechas donde hay más tweets. Ordenadas por usuario (username) que más publicaciones tiene por cada uno de esos días: ")
print(output)

QUERY 1
Las top 10 fechas donde hay más tweets. Ordenadas por usuario (username) que más publicaciones tiene por cada uno de esos días: 
[('RanbirS00614606', '2021-02-12'), ('MaanDee08215437', '2021-02-13'), ('RaaJVinderkaur', '2021-02-17'), ('jot__b', '2021-02-16'), ('rebelpacifist', '2021-02-14'), ('neetuanjle_nitu', '2021-02-18'), ('jot__b', '2021-02-15'), ('MangalJ23056160', '2021-02-20'), ('Surrypuria', '2021-02-23'), ('Preetm91', '2021-02-19')]


In [9]:
query1= """
  WITH datos AS (
    SELECT
      FORMAT_TIMESTAMP('%Y-%m-%d', TIMESTAMP(date)) AS fecha_formateada,
      user.username as usuario
    FROM
      `tw-techdash.de_challenge_latam.tweets_latam`
  ),

  conteo_por_fecha AS (
    SELECT
      fecha_formateada,
      COUNT(usuario) AS num_tweets
    FROM
      datos
    GROUP BY
      fecha_formateada
    ORDER BY
      num_tweets DESC
    LIMIT 10
  ),

  usuario_top_por_fecha AS (
    SELECT
      fecha_formateada,
      usuario,
      COUNT(usuario) AS num_tweets_por_usuario,
      RANK() OVER (PARTITION BY fecha_formateada ORDER BY COUNT(usuario) DESC) AS rank
    FROM
      datos
    WHERE
      fecha_formateada IN (SELECT fecha_formateada FROM conteo_por_fecha)
    GROUP BY
      fecha_formateada, usuario
  )

  SELECT
    u.usuario AS top_user,
    u.fecha_formateada as date,
  FROM
    usuario_top_por_fecha u
  JOIN
    conteo_por_fecha c
  ON
    u.fecha_formateada = c.fecha_formateada
  WHERE
    u.rank = 1
  ORDER BY
    c.num_tweets DESC
"""

In [20]:
%run q1_memory.py
q1_memory

<function __main__.q1_memory(client: google.cloud.bigquery.client.Client, query: str) -> List[Tuple[str, str]]>

In [29]:
q1_memory(client, query1)

Filename: /content/drive/Othercomputers/My MacBook Pro/challenge_DE/src/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     6    207.4 MiB    207.4 MiB           1   @profile
     7                                         def q1_memory(client: bigquery.Client, query: str) ->  List[Tuple[str, str]]:
     8                                         
     9    207.4 MiB      0.0 MiB           1       query_job = client.query(query)
    10    207.4 MiB      0.0 MiB           1       query_job.result()
    11                                         
    12    207.4 MiB      0.0 MiB           1       results = query_job.result()
    13                                         
    14    207.4 MiB      0.0 MiB          13       output = [(row.top_user, row.date) for row in results]
    15                                         
    16    207.4 MiB      0.0 MiB           1       return output




[('RanbirS00614606', '2021-02-12'),
 ('MaanDee08215437', '2021-02-13'),
 ('RaaJVinderkaur', '2021-02-17'),
 ('jot__b', '2021-02-16'),
 ('rebelpacifist', '2021-02-14'),
 ('neetuanjle_nitu', '2021-02-18'),
 ('jot__b', '2021-02-15'),
 ('MangalJ23056160', '2021-02-20'),
 ('Surrypuria', '2021-02-23'),
 ('Preetm91', '2021-02-19')]

In [32]:
%run q1_time.py
q1_time

[autoreload of time_utils failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.10/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/content/drive/Othercomputers/My MacBook Pro/challenge_DE/src/time_utils.py", line 7, in <module>
    def execute_time_query(client: bigquery.Client, query: str) -> Tuple[Any, float]:
NameError: name 'Any' is not defined
]


<function __main__.q1_time(client: google.cloud.bigquery.client.Client, query: str) -> Tuple[List[Tuple[str, str]], float]>

In [33]:
q1_time(client, query1)

         9058 function calls (8989 primitive calls) in 0.613 seconds

   Ordered by: cumulative time
   List reduced from 597 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      4/3    0.000    0.000    0.612    0.204 retry_unary.py:286(retry_wrapped_func)
      4/3    0.000    0.000    0.612    0.204 retry_unary.py:85(retry_target)
        2    0.000    0.000    0.611    0.306 client.py:814(_call_api)
        2    0.000    0.000    0.611    0.305 __init__.py:383(api_request)
        2    0.000    0.000    0.611    0.305 __init__.py:276(_make_request)
        2    0.000    0.000    0.610    0.305 __init__.py:345(_do_request)
        2    0.000    0.000    0.610    0.305 requests.py:482(request)
        2    0.000    0.000    0.610    0.305 sessions.py:500(request)
        2    0.000    0.000    0.606    0.303 sessions.py:673(send)
        2    0.000    0.000    0.605    0.303 adapters.py:613(send)


Tiempo total de ejecución: 0.6

([('RanbirS00614606', '2021-02-12'),
  ('MaanDee08215437', '2021-02-13'),
  ('RaaJVinderkaur', '2021-02-17'),
  ('jot__b', '2021-02-16'),
  ('rebelpacifist', '2021-02-14'),
  ('neetuanjle_nitu', '2021-02-18'),
  ('jot__b', '2021-02-15'),
  ('MangalJ23056160', '2021-02-20'),
  ('Surrypuria', '2021-02-23'),
  ('Preetm91', '2021-02-19')],
 0.6127767562866211)

In [39]:
query2 = """
  SELECT
  emoji,
  COUNT(*) AS total_emoji
FROM
  (SELECT
        REGEXP_EXTRACT_ALL(
          content,
           r"[\p{So}]"
        ) AS emojis,
        id
      FROM
        `tw-techdash.de_challenge_latam.tweets_latam`
      WHERE id is not null
  )
CROSS JOIN UNNEST(emojis) as emoji
WHERE emoji != '\uFFFD' -- �
group by emoji
order by total_emoji desc
limit 10




"""

In [45]:
%run q2_time.py
q2_time

<function __main__.q2_time(client: google.cloud.bigquery.client.Client, query: str) -> Tuple[List[Tuple[str, int]], float]>

In [46]:
q2_time(client, query2)

         9372 function calls (9305 primitive calls) in 0.480 seconds

   Ordered by: cumulative time
   List reduced from 550 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      4/3    0.000    0.000    0.479    0.160 retry_unary.py:286(retry_wrapped_func)
      4/3    0.000    0.000    0.479    0.160 retry_unary.py:85(retry_target)
        2    0.000    0.000    0.479    0.240 client.py:814(_call_api)
        2    0.000    0.000    0.479    0.239 __init__.py:383(api_request)
        2    0.000    0.000    0.478    0.239 __init__.py:276(_make_request)
        2    0.000    0.000    0.478    0.239 __init__.py:345(_do_request)
        2    0.000    0.000    0.478    0.239 requests.py:482(request)
        2    0.000    0.000    0.478    0.239 sessions.py:500(request)
        2    0.000    0.000    0.474    0.237 sessions.py:673(send)
        2    0.000    0.000    0.473    0.236 adapters.py:613(send)


Tiempo total de ejecución: 0.4

([('✊', 2411),
  ('❤', 1779),
  ('☮', 316),
  ('✌', 274),
  ('♂', 184),
  ('♀', 148),
  ('✍', 103),
  ('♥', 81),
  ('⛳', 68),
  ('✅', 57)],
 0.4803769588470459)

In [49]:
query3 = """
 WITH gruped AS (
  SELECT
    user.username AS username,
    m.username AS top_mentioned,
    COUNT(*) AS cuenta
  FROM
    `tw-techdash.de_challenge_latam.tweets_latam` as tweets,
    UNNEST(mentionedUsers) AS m
  WHERE tweets.id is not null
  GROUP BY
    username, top_mentioned
)

SELECT
  top_mentioned,
  SUM(cuenta) AS total_mencioned
FROM
  gruped
GROUP BY
  top_mentioned
  order by total_mencioned desc
LIMIT 10





"""

In [47]:
%run q3_time.py
q3_time

<function __main__.q3_time(client: google.cloud.bigquery.client.Client, query: str) -> Tuple[List[Tuple[str, int]], float]>

In [50]:
q3_time(client, query3)

         9037 function calls (8970 primitive calls) in 0.892 seconds

   Ordered by: cumulative time
   List reduced from 550 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      4/3    0.000    0.000    0.892    0.297 retry_unary.py:286(retry_wrapped_func)
      4/3    0.000    0.000    0.892    0.297 retry_unary.py:85(retry_target)
        2    0.000    0.000    0.891    0.446 client.py:814(_call_api)
        2    0.000    0.000    0.891    0.446 __init__.py:383(api_request)
        2    0.000    0.000    0.891    0.445 __init__.py:276(_make_request)
        2    0.000    0.000    0.891    0.445 __init__.py:345(_do_request)
        2    0.000    0.000    0.891    0.445 requests.py:482(request)
        2    0.000    0.000    0.890    0.445 sessions.py:500(request)
        2    0.000    0.000    0.887    0.444 sessions.py:673(send)
        2    0.000    0.000    0.886    0.443 adapters.py:613(send)


Tiempo total de ejecución: 0.8

([('narendramodi', 2265),
  ('Kisanektamorcha', 1840),
  ('RakeshTikaitBKU', 1644),
  ('PMOIndia', 1427),
  ('RahulGandhi', 1146),
  ('GretaThunberg', 1048),
  ('RaviSinghKA', 1019),
  ('rihanna', 986),
  ('UNHumanRights', 962),
  ('meenaharris', 926)],
 0.8925361633300781)

In [14]:
import time

query = """
WITH datos AS (
  SELECT
    FORMAT_TIMESTAMP('%Y-%m-%d', TIMESTAMP(date)) AS fecha_formateada,
    user.username as usuario
  FROM
    `tw-techdash.de_challenge_latam.tweets_latam`
),

conteo_por_fecha AS (
  SELECT
    fecha_formateada,
    COUNT(usuario) AS num_tweets
  FROM
    datos
  GROUP BY
    fecha_formateada
  ORDER BY
    num_tweets DESC
  LIMIT 10
),

usuario_top_por_fecha AS (
  SELECT
    fecha_formateada,
    usuario,
    COUNT(usuario) AS num_tweets_por_usuario,
    RANK() OVER (PARTITION BY fecha_formateada ORDER BY COUNT(usuario) DESC) AS rank
  FROM
    datos
  WHERE
    fecha_formateada IN (SELECT fecha_formateada FROM conteo_por_fecha)
  GROUP BY
    fecha_formateada, usuario
)

SELECT
  u.usuario AS top_user,
  u.fecha_formateada as date,
FROM
  usuario_top_por_fecha u
JOIN
  conteo_por_fecha c
ON
  u.fecha_formateada = c.fecha_formateada
WHERE
  u.rank = 1
ORDER BY
  c.num_tweets DESC;

"""

start_time = time.time()

query_job = client.query(query)

query_job.result()

results = query_job.result()

output = [(row.top_user, row.date) for row in results]

end_time = time.time()
execution_time = end_time - start_time

print("QUERY 1")
print ("Las top 10 fechas donde hay más tweets. Ordenadas por usuario (username) que más publicaciones tiene por cada uno de esos días: ")
print(output)
print(f"Tiempo total de ejecución: {execution_time:.2f} segundos")

QUERY 1
Las top 10 fechas donde hay más tweets. Ordenadas por usuario (username) que más publicaciones tiene por cada uno de esos días: 
[('RanbirS00614606', '2021-02-12'), ('MaanDee08215437', '2021-02-13'), ('RaaJVinderkaur', '2021-02-17'), ('jot__b', '2021-02-16'), ('rebelpacifist', '2021-02-14'), ('neetuanjle_nitu', '2021-02-18'), ('jot__b', '2021-02-15'), ('MangalJ23056160', '2021-02-20'), ('Surrypuria', '2021-02-23'), ('Preetm91', '2021-02-19')]
Tiempo total de ejecución: 0.63 segundos


In [15]:
import cProfile
import pstats

import time

# Definir la consulta SQL
query = """
WITH datos AS (
  SELECT
    FORMAT_TIMESTAMP('%Y-%m-%d', TIMESTAMP(date)) AS fecha_formateada,
    user.username as usuario
  FROM
    `tw-techdash.de_challenge_latam.tweets_latam`
),

conteo_por_fecha AS (
  SELECT
    fecha_formateada,
    COUNT(usuario) AS num_tweets
  FROM
    datos
  GROUP BY
    fecha_formateada
  ORDER BY
    num_tweets DESC
  LIMIT 10
),

usuario_top_por_fecha AS (
  SELECT
    fecha_formateada,
    usuario,
    COUNT(usuario) AS num_tweets_por_usuario,
    RANK() OVER (PARTITION BY fecha_formateada ORDER BY COUNT(usuario) DESC) AS rank
  FROM
    datos
  WHERE
    fecha_formateada IN (SELECT fecha_formateada FROM conteo_por_fecha)
  GROUP BY
    fecha_formateada, usuario
)

SELECT
  u.usuario AS top_user,
  u.fecha_formateada as date
FROM
  usuario_top_por_fecha u
JOIN
  conteo_por_fecha c
ON
  u.fecha_formateada = c.fecha_formateada
WHERE
  u.rank = 1
ORDER BY
  c.num_tweets DESC;
"""

def run_query():
    query_job = client.query(query)
    query_job.result()
    results = query_job.result()
    output = [(row.top_user, row.date) for row in results]

    print("QUERY 1")
    print("Las top 10 fechas donde hay más tweets. Ordenadas por usuario (username) que más publicaciones tiene por cada uno de esos días:")
    print(output)

# Medir el tiempo total de ejecución y usar cProfile para obtener detalles
profiler = cProfile.Profile()
profiler.enable()

start_time = time.time()

# Ejecutar la consulta
run_query()

end_time = time.time()
execution_time = end_time - start_time

profiler.disable()

# Mostrar las estadísticas del profiler
stats = pstats.Stats(profiler)
stats.strip_dirs().sort_stats('cumtime').print_stats(10)  # Mostrar las 10 líneas más lentas

print(f"Tiempo total de ejecución: {execution_time:.2f} segundos")


QUERY 1
Las top 10 fechas donde hay más tweets. Ordenadas por usuario (username) que más publicaciones tiene por cada uno de esos días:
[('RanbirS00614606', '2021-02-12'), ('MaanDee08215437', '2021-02-13'), ('RaaJVinderkaur', '2021-02-17'), ('jot__b', '2021-02-16'), ('rebelpacifist', '2021-02-14'), ('neetuanjle_nitu', '2021-02-18'), ('jot__b', '2021-02-15'), ('MangalJ23056160', '2021-02-20'), ('Surrypuria', '2021-02-23'), ('Preetm91', '2021-02-19')]
         14753 function calls (14681 primitive calls) in 0.674 seconds

   Ordered by: cumulative time
   List reduced from 614 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.000    0.000    0.674    0.135 interactiveshell.py:3512(run_code)
        5    0.000    0.000    0.673    0.135 {built-in method builtins.exec}
        1    0.000    0.000    0.673    0.673 <ipython-input-15-6cea65cd9e77>:1(<cell line: 75>)
        1    0.000    0.000    0.673    0.673 <ipython-inpu